<a href="https://colab.research.google.com/github/iskra3138/stable-baselines/blob/main/CartPole_Dynamic_Change_DQN_Stable_Baselines.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Setup


In [1]:
# Stable Baselines only supports tensorflow 1.x for now
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [ ]:
!apt-get install ffmpeg freeglut3-dev xvfb  # For visualization
!pip install stable-baselines[mpi]==2.10.0

In [2]:
import numpy as np
import math
import gym

In [3]:
from gym.envs.classic_control import cartpole

class CartPoleEnv_stochastic(cartpole.CartPoleEnv):

  def __init__(self, mu, sigma):
    super(CartPoleEnv_stochastic, self).__init__()
    self.mu = mu
    self.sigma = sigma

  def step(self, action):
    err_msg = "%r (%s) invalid" % (action, type(action))
    assert self.action_space.contains(action), err_msg

    x, x_dot, theta, theta_dot = self.state
    ### to replace deterministic action with stochastic action 
    #force = self.force_mag if action == 1 else -self.force_mag
    stochastic_force = self.force_mag + np.random.normal(self.mu, self.sigma)
    force = stochastic_force if action == 1 else -stochastic_force

    costheta = math.cos(theta)
    sintheta = math.sin(theta)

    # For the interested reader:
    # https://coneural.org/florian/papers/05_cart_pole.pdf
    temp = (force + self.polemass_length * theta_dot ** 2 * sintheta) / self.total_mass
    thetaacc = (self.gravity * sintheta - costheta * temp) / (self.length * (4.0 / 3.0 - self.masspole * costheta ** 2 / self.total_mass))
    xacc = temp - self.polemass_length * thetaacc * costheta / self.total_mass

    if self.kinematics_integrator == 'euler':
        x = x + self.tau * x_dot
        x_dot = x_dot + self.tau * xacc
        theta = theta + self.tau * theta_dot
        theta_dot = theta_dot + self.tau * thetaacc
    else:  # semi-implicit euler
        x_dot = x_dot + self.tau * xacc
        x = x + self.tau * x_dot
        theta_dot = theta_dot + self.tau * thetaacc
        theta = theta + self.tau * theta_dot

    self.state = (x, x_dot, theta, theta_dot)

    done = bool(
        x < -self.x_threshold
        or x > self.x_threshold
        or theta < -self.theta_threshold_radians
        or theta > self.theta_threshold_radians
    )

    if not done:
        reward = 1.0
    elif self.steps_beyond_done is None:
        # Pole just fell!
        self.steps_beyond_done = 0
        reward = 1.0
    else:
        if self.steps_beyond_done == 0:
            logger.warn(
                "You are calling 'step()' even though this "
                "environment has already returned done = True. You "
                "should always call 'reset()' once you receive 'done = "
                "True' -- any further steps are undefined behavior."
            )
        self.steps_beyond_done += 1
        reward = 0.0

    return np.array(self.state), reward, done, {}

In [4]:
# To check deterministic actiong
environment = CartPoleEnv_stochastic(mu=0.0, sigma=0.0)
for _ in range(3):
  environment.seed(1)
  step = environment.reset()
  next_step = environment.step(np.array(0, dtype=np.int32))
  print (step, '- >', next_step[0])

[ 0.03073904  0.00145001 -0.03088818 -0.03131252] - > [ 0.03076804 -0.19321569 -0.03151444  0.25146705]
[ 0.03073904  0.00145001 -0.03088818 -0.03131252] - > [ 0.03076804 -0.19321569 -0.03151444  0.25146705]
[ 0.03073904  0.00145001 -0.03088818 -0.03131252] - > [ 0.03076804 -0.19321569 -0.03151444  0.25146705]


In [5]:
# To check stochastic action
environment = CartPoleEnv_stochastic(mu=0.0, sigma=0.5)
for _ in range(3):
  environment.seed(1)
  step = environment.reset()
  next_step = environment.step(np.array(0, dtype=np.int32))
  print (step, '- >', next_step[0])

[ 0.03073904  0.00145001 -0.03088818 -0.03131252] - > [ 0.03076804 -0.18513334 -0.03151444  0.23934932]
[ 0.03073904  0.00145001 -0.03088818 -0.03131252] - > [ 0.03076804 -0.20348305 -0.03151444  0.26686076]
[ 0.03073904  0.00145001 -0.03088818 -0.03131252] - > [ 0.03076804 -0.18848618 -0.03151444  0.24437618]


In [6]:
env = CartPoleEnv_stochastic(mu=0.0, sigma=0.5)
env = gym.wrappers.TimeLimit(env, max_episode_steps=500)

In [ ]:
# Same as before we instantiate the agent along with the environment
from stable_baselines import DQN

# Deactivate all the DQN extensions to have the original version
# In practice, it is recommend to have them activated
# 본 예제에서는 모든 확장 기능을 비활성화시킴. 실제에서는 모두 활성화하길 추천함
kwargs = {'double_q': False, 'prioritized_replay': True, 'policy_kwargs': dict(dueling=False)}

# Note that the MlpPolicy of DQN is different from the one of PPO
# but stable-baselines handles that automatically if you pass a string
# DQN의 MlpPolicy는 PPO의 그것과 다르지만 string argument를 이용해서 자동으로 적절할 MlpPolicy가 선택됨
dqn_model = DQN('MlpPolicy', env, verbose=1, learning_rate=1e-3, batch_size=64, buffer_size=100000, **kwargs)

In [9]:
def evaluate(model, env, num_episodes=10):
    """
    Evaluate a RL agent
    :param model: (BaseRLModel object) the RL Agent
    :param num_episodes: (int) number of episodes to evaluate it
    :return: (float) Mean reward for the last num_episodes
    """
    # This function will only work for a single Environment
    # single Environment에 대해서만 작동함
    #env = model.get_env() 
    all_episode_rewards = []
    for i in range(num_episodes):
        episode_rewards = []
        done = False
        obs = env.reset() # e.g. [[ 0.04632042, -0.01863058, -0.03169358,  0.00881829]])
        while not done:
            # _states are only useful when using LSTM policies
            action, _states = model.predict(obs, deterministic=True) # e.g.: [0], None
            # here, action, rewards and dones are arrays
            # because we are using vectorized env
            obs, reward, done, info = env.step(action) #e.g.: [[0.04, -0.24, -0.03, 0.27]], [1.0], [False], [{}]
            episode_rewards.append(reward)

        all_episode_rewards.append(sum(episode_rewards))

    mean_episode_reward = np.mean(all_episode_rewards)
    print("Mean reward:", mean_episode_reward, "Num episodes:", num_episodes)

    return mean_episode_reward

In [13]:
# Random Agent, before training
eval_env = gym.wrappers.TimeLimit(CartPoleEnv_stochastic(mu=0.0, sigma=0.0), max_episode_steps=500)
eval_env.seed(1)
mean_reward_before_train = evaluate(dqn_model, eval_env, num_episodes=1)

Mean reward: 104.0 Num episodes: 1


In [ ]:
# Train the agent for 10000 steps
dqn_model.learn(total_timesteps=1000000, log_interval=10)

In [28]:
# Evaluate the trained agent
eval_env = CartPoleEnv_stochastic(mu=0.0, sigma=0.0)
eval_env = gym.wrappers.TimeLimit(eval_env, max_episode_steps=500)
eval_env.seed(1)

mean_reward = evaluate(dqn_model, eval_env, num_episodes=1)

Mean reward: 500.0 Num episodes: 1


In [25]:
from stable_baselines.common.evaluation import evaluate_policy
eval_env = CartPoleEnv_stochastic(mu=0.0, sigma=0.0)
eval_env = gym.wrappers.TimeLimit(eval_env, max_episode_steps=500)
eval_env.seed(1)

mean_reward, std_reward = evaluate_policy(dqn_model, eval_env, n_eval_episodes=1)

print(f"mean_reward:{mean_reward:.2f} +/- {std_reward:.2f}")

mean_reward:500.00 +/- 0.00


In [29]:
# mu=0.0으로 고정
sigmas = [0.0, 1.0, 3.0, 5.0, 10.0]
for sigma in sigmas :
  eval_env = CartPoleEnv_stochastic(mu=0.0, sigma=sigma)
  eval_env = gym.wrappers.TimeLimit(eval_env, max_episode_steps=500)
  eval_env.seed(1)

  print (sigma, 'mean_reward = ', evaluate(dqn_model, eval_env, num_episodes=10))

Mean reward: 308.9 Num episodes: 10
0.0 mean_reward =  308.9
Mean reward: 215.3 Num episodes: 10
1.0 mean_reward =  215.3
Mean reward: 213.2 Num episodes: 10
3.0 mean_reward =  213.2
Mean reward: 21.3 Num episodes: 10
5.0 mean_reward =  21.3
Mean reward: 14.8 Num episodes: 10
10.0 mean_reward =  14.8


### Prepare video recording

In [19]:
# Set up fake display; otherwise rendering will fail
import os
os.system("Xvfb :1 -screen 0 1024x768x24 &")
os.environ['DISPLAY'] = ':1'

In [20]:
import base64
from pathlib import Path

from IPython import display as ipythondisplay

def show_videos(video_path='', prefix=''):
  """
  Taken from https://github.com/eleurent/highway-env

  :param video_path: (str) Path to the folder containing videos
  :param prefix: (str) Filter the video, showing only the only starting with this prefix
  """
  html = []
  for mp4 in Path(video_path).glob("{}*.mp4".format(prefix)):
      video_b64 = base64.b64encode(mp4.read_bytes())
      html.append('''<video alt="{}" autoplay 
                    loop controls style="height: 400px;">
                    <source src="data:video/mp4;base64,{}" type="video/mp4" />
                </video>'''.format(mp4, video_b64.decode('ascii')))
  ipythondisplay.display(ipythondisplay.HTML(data="<br>".join(html)))

We will record a video using the [VecVideoRecorder](https://stable-baselines.readthedocs.io/en/master/guide/vec_envs.html#vecvideorecorder) wrapper, you will learn about those wrapper in the next notebook.

In [21]:
from stable_baselines.common.vec_env import VecVideoRecorder, DummyVecEnv

def record_video(env, model, video_length=500, prefix='', video_folder='videos/'):
  """
  :param env_id: (str)
  :param model: (RL model)
  :param video_length: (int)
  :param prefix: (str)
  :param video_folder: (str)
  """
  eval_env = DummyVecEnv([lambda: env])
  # Start the video at step=0 and record 500 steps
  eval_env = VecVideoRecorder(eval_env, video_folder=video_folder,
                              record_video_trigger=lambda step: step == 0, video_length=video_length,
                              name_prefix=prefix)

  obs = eval_env.reset()
  for _ in range(video_length):
    action, _ = model.predict(obs, deterministic=True)
    obs, _, _, _ = eval_env.step(action)

  # Close the video recorder
  eval_env.close()

### Visualize trained agent



In [22]:
record_video(env, dqn_model, video_length=500, prefix='dqn-cartpole')

Saving video to  /content/videos/dqn-cartpole-step-0-to-step-500.mp4


In [23]:
show_videos('videos', prefix='dqn')